In [ ]:
import pandas as pd

df_org = pd.read_csv("../rawData/all_data_08_18.csv")
df_org

In [ ]:
from datetime import datetime


# manual parsing of time
def parse_time(s):
    try:
        return datetime.strptime(s, "%Y-%m-%d %H:%M:%S.%f")
    except ValueError:
        try:
            return datetime.strptime(s, "%Y-%m-%d %H:%M:%S.%f")
        except ValueError:
            return datetime.strptime(s, "%Y-%m-%d")  # midnight fallback

In [ ]:
def resample_data(fileName):
    # filter to needed data
    person = df_org[df_org.file == fileName].copy()
    person = person.drop(["Unnamed: 0", "timestamp", "file"], axis=1)

    person["datetime"] = pd.to_datetime(person["datetime"], errors="coerce")
    # person["datetime"] = person["datetime"].apply(parse_time)

    # resampte to hourly data
    resample_person = person.resample("h", on="datetime").mean()

    # interpolate missing values
    resample_person_fill = resample_person.interpolate()

    # path = "processedData/all_resampled_" + fileName
    # resample_person_fill.to_csv(path)
    return resample_person_fill

In [ ]:
persons = df_org.file.unique()
persons

In [ ]:
resample_data("Annapurna.csv")

In [ ]:
all_data = []
for p in persons:
    print(p)
    resampled = resample_data(p)
    resampled["person"] = p
    all_data.append(resampled)

combined_data = pd.concat(all_data, axis=0)

In [ ]:
combined_data.to_csv("./processedData/all_resampled_08_18.csv")

In [ ]:
combined_data

In [ ]:
# combine humidity + temperature
# https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.heat_index.html

from metpy.calc import heat_index
from metpy.units import units


def safe_heat_index(temp_c, rh):
    # calc heat temp
    hi = heat_index(temp_c * units.degC, rh * units.percent).to("degC")
    # return heat index, or normal temp if temp below 26.9 degrees
    if hasattr(hi.magnitude, "mask"):
        if hi.magnitude.mask:  # masked → invalid
            return temp_c
        else:  # valid → get value
            return hi.magnitude.item()
    else:
        # Normal numpy scalar
        return hi.magnitude.item()


df = combined_data.copy()
# df["heat_index"] = df.apply(lambda row: heat_index(row["temperature"] * units.degC, row["humidity"] * units.percent).to("degC").magnitude,axis=1)
# df["heat_index_f"] = heat_index(df.temperature.values * units.degC, df.humidity.values * units.percent).to(units.degC)
df["heat_index_c"] = df.apply(lambda row: safe_heat_index(row["temperature"], row["humidity"]), axis=1)
# df["heat_index_c"]  = df["heat_index"].magnitude
df.dtypes

In [ ]:
df

In [ ]:
df.to_csv("./processedData/all_resampled_08_08_heatindex.csv")